In [1]:
import camelot
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from datetime import timedelta, date
import warnings
warnings.filterwarnings('ignore')
# DATE TIME RANGER CREATOR
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
import os, sys, traceback

In [81]:
def get_data_TimeSeries_KeyIndicators(file):
    table_1_read = camelot.read_pdf(file, flavor='stream', table_areas=['0,680,400,500'],strip_text='*^')
    temp=table_1_read[0].df
    lookup_stop_header=temp[temp[0]=="ICT"].index.values.astype(int)[0]
    temp.columns=temp.loc[0:lookup_stop_header-1].fillna('').apply(' '.join).str.strip()
    lookup_start=temp[temp['Region']=="ICT"].index.values.astype(int)[0]
    lookup_stop=temp[temp['Region']=="Total"].index.values.astype(int)[0]
    table_1=temp[lookup_start:lookup_stop].replace("-",0).replace('',0)
    table_1
    table_1['Date']=Date
    table_1=table_1.replace(',','', regex=True).apply(pd.to_numeric,errors='ignore')
    ignore_rows = table_1['Region'].isin(['Taftan', 'Blochistan',0])
    table_1=table_1[~ignore_rows]
    # print(table_1.dtypes)
    return table_1

def get_data_TimeSeries_Action_Screen(file):
    table_2_read = camelot.read_pdf(file, flavor='stream', table_areas=['400,710,600,670'],strip_text='*\n')
    # camelot.plot(table_23_read[0], kind='contour')
    table_2=table_2_read[0].df
#     table_2=table_2.loc[:,[0,2]][0:4]
    table_2=table_2.replace("",np.nan)
    table_2=table_2.ffill(axis = 0)
    table_2.columns=['Date','Values']
    table_2=table_2.groupby(['Values'], as_index=False).sum()
    table_2_headers=table_2[['Date','Values']].T.iloc[0]
    table_2=table_2[['Date','Values']].T[1:]
    table_2.columns=table_2_headers
    table_2.rename(index={'Values':Date}, inplace=True)
    table_2=table_2.replace(',','', regex=True).apply(pd.to_numeric,errors='ignore')
    return table_2

def get_data_TimeSeries_Action_Call(file):
    table_3_read = camelot.read_pdf(file, flavor='stream', table_areas=['400,670,600,460'],strip_text='*\n')
    table_3=table_3_read[0].df
    lookup_start=table_3[table_3[0]=="Call Records (n)"].index.values.astype(int)[0]
    lookup_stop=table_3[table_3[0]=="Quarantine"].index.values.astype(int)[0]+1
    table_3=table_3[lookup_start:lookup_stop].reset_index(drop=True)
    table_3.columns=table_3.loc[0:1].apply(' '.join).str.strip()
    table_3=table_3[2:].reset_index(drop=True)#.replace("",np.nan)
    table_3['index']=[val for val in range(1,6) for _ in (0, 1)]
    # len([val for val in range(1,6) for _ in (0, 1)])
    table_3=table_3.groupby(['index'], as_index=False).sum()
    table_3=table_3[['Call Records (n)','Returnees from Iran','Returnees from China']]
    header=table_3.T.iloc[0]
    table_3=table_3.T[1:]
    table_3.columns=header
    table_3['Date']=Date
    table_3=table_3.replace(',','', regex=True).apply(pd.to_numeric,errors='ignore')
    return table_3
    
def get_data_TimeSeries_COVID_News(file):
    #READING LOWER REGION DATA
    table_4_read = camelot.read_pdf(file, flavor='stream', table_areas=['000,580,320,200'],strip_text='*\n')
    # camelot.plot(table_4_read[0], kind='contour')
    temp=table_4_read[0].df
    col_split=len(temp.columns)-1


    #DAILY NEWS ON CORONA
    lookup_start=temp[temp[0]=="Updates: Pakistan:"].index.values.astype(int)[0]
    lookup_stop=temp[temp[0]=="NIH Preparedness and Response:"].index.values.astype(int)[0]
    temp_2=temp[lookup_start:lookup_stop][[0,col_split]].replace("",np.nan).replace("•",np.nan)
    temp_2.columns=['val1','val2']
    temp_2['News']=temp_2['val1'].fillna(temp_2['val2'])
    temp_2=temp_2.reset_index(drop=True)
    temp_2['Date']=Date
    temp_2=temp_2[['Date','News']]
    temp_2['News']=temp_2['News'].replace(u'\uf0b7','•',regex=True)
    for idx,i in enumerate(temp_2['News']):
        if temp_2['News'][idx][-1]!="•" and temp_2['News'][idx][-1]!=":":
            temp_2['News'][idx]=temp_2['News'][idx]+" "+"•"
            
        if temp_2['News'][idx][-1]!="•" and temp_2['News'][idx][-1]!=":":
    #         print(temp_2['News'][idx])
            temp_2['News'][idx-1]= (temp_2['News'][idx-1]+" "+temp_2['News'][idx]).replace("•","")+" •"
            temp_2['News'][idx]=""
    temp_2=temp_2.replace("",np.nan).ffill(axis=0)
    # temp_2
    table_4=temp_2[['Date','News']].drop_duplicates()
    merge_temp=table_4.groupby(["Date"])["News"]\
        .apply(' \n '.join)\
        .reset_index()\
        .rename(columns={'News':'Agg News'})
    table_4=pd.merge(table_4,merge_temp,how='left', on=["Date"])  
    return table_4

def get_data_TimeSeries_NIH_Response(file):
    #READING LOWER REGION DATA
    table_4_read = camelot.read_pdf(file, flavor='stream', table_areas=['000,580,320,200'],strip_text='*\n')
    # camelot.plot(table_4_read[0], kind='contour')
    temp=table_4_read[0].df
    col_split=len(temp.columns)-1

    #NIH PREPAREDNESS & RESPONSE
    lookup_start=temp[temp[0]=="NIH Preparedness and Response:"].index.values.astype(int)[0]
    lookup_stop=temp[temp[0].str.contains("Risk Assessment:")].index.values.astype(int)[0]
    temp_2=temp[lookup_start:lookup_stop][[0,col_split]]
    temp_2.columns=['val1','val2']
    if all(temp_2['val2']!=temp_2['val1']):
        temp_2['NIH Preparedness and Response']=temp_2['val2']+" "+temp_2['val1']
    else:
        temp_2['NIH Preparedness and Response']=temp_2['val2']
    temp_2=temp_2[1:].reset_index(drop=True)
    temp_2['NIH Preparedness and Response']=temp_2['NIH Preparedness and Response'].replace(u'\uf0b7','•',regex=True)
    for idx,i in enumerate(temp_2['NIH Preparedness and Response']): 
        if temp_2['NIH Preparedness and Response'][idx][-1]!="•":
#             print(temp_2['NIH Preparedness and Response'][idx])
            temp_2['NIH Preparedness and Response'][idx-1]= (temp_2['NIH Preparedness and Response'][idx-1]+" "+temp_2['NIH Preparedness and Response'][idx]).replace("•","")+" •"
            temp_2['NIH Preparedness and Response'][idx]=""
    temp_2=temp_2.replace("",np.nan).ffill(axis=0)
    temp_2['Date']=Date
    table_5=temp_2[['Date','NIH Preparedness and Response']].drop_duplicates()
    merge_temp=table_5.groupby(["Date"])["NIH Preparedness and Response"]\
        .apply(' \n '.join)\
        .reset_index()\
        .rename(columns={'NIH Preparedness and Response':'Agg NIH Preparedness and Response'})
    table_5=pd.merge(table_5,merge_temp,how='left', on=["Date"])  
    return table_5
    
def get_data_TimeSeries_NIH_Risk(file):
    #READING LOWER REGION DATA
    table_4_read = camelot.read_pdf(file, flavor='stream', table_areas=['000,580,320,200'],strip_text='*\n')
    # camelot.plot(table_4_read[0], kind='contour')
    temp=table_4_read[0].df
    col_split=len(temp.columns)-1

    # DAILY RISK ASSESSMENT
    lookup_val=temp[temp[0].str.contains("Risk Assessment:")][0].reset_index(drop=True)[0]
    temp_4=pd.DataFrame(lookup_val.split(":")).T[[2]]
    temp_4.columns=['Risk Assessment NIH']
    temp_4['Date']=Date
    table_6=temp_4[['Date','Risk Assessment NIH']]
#     table_6=table_6.apply(pd.to_numeric,errors='ignore')
    return table_6
    
def get_data_TimeSeries_Province_Response(file):
    #PROVINCES PREPAREDNESS & RESPONSE
    if Date=='19-Mar-2020':
        table_7_read = camelot.read_pdf(file, flavor='stream', table_areas=['000,450,400,270'],strip_text='*\n')
    else:
        table_7_read = camelot.read_pdf(file, flavor='stream', table_areas=['000,450,400,240'],strip_text='*\n')
    temp_3=table_7_read[0].df
#     camelot.plot(table_7_read[0], kind='contour')
    lookup_start=temp_3[temp_3[0]=="Provinces Preparedness and Response"].index.values.astype(int)[0]
    lookup_stop=temp_3[temp_3[0].isin(["Risk Communication:", "Risk Communication Message:"])].index.values.astype(int)[0]

    temp_3=temp_3[lookup_start+1:lookup_stop]#.ffill(axis=0)

    col_split=len(temp_3.columns)-1

    if col_split>1:
        temp_3[1]=temp_3[1]
        temp_3[2]=temp_3[2]+" "+temp_3[1]
    temp_3=temp_3[[0,col_split]].reset_index(drop=True)
    temp_3.columns=['region','Provinces Preparedness and Response']
    temp_3

    # MERGING ALL THE NEW LINE ROWS INTO ONE
    temp_3['Provinces Preparedness and Response']=temp_3['Provinces Preparedness and Response'].replace(u'\uf0b7','•',regex=True)
    for idx,i in enumerate(temp_3['Provinces Preparedness and Response']): 
        if "•" in temp_3['Provinces Preparedness and Response'][idx]:
            temp_3['Provinces Preparedness and Response'][idx]=temp_3['Provinces Preparedness and Response'][idx].replace("•","")+" "+"•"
        if temp_3['Provinces Preparedness and Response'][idx][-1]!="•":
            temp_3['Provinces Preparedness and Response'][idx-1]= (temp_3['Provinces Preparedness and Response'][idx-1]+" "+temp_3['Provinces Preparedness and Response'][idx]).replace("•","")+" •"
            temp_3['Provinces Preparedness and Response'][idx]=""
    table_7=temp_3.replace("",np.nan).ffill(axis=0).drop_duplicates()
    table_7['Date']=Date
    merge_temp=table_7.groupby(["Date","region"])["Provinces Preparedness and Response"]\
        .apply(' \n '.join)\
        .reset_index()\
        .rename(columns={'Provinces Preparedness and Response':'Agg Provinces Preparedness and Response'})
    table_7=pd.merge(table_7,merge_temp,how='left', on=["Date","region"])   
    return table_7 

def get_data (file):
    global table_1,table_2,table_3,table_4,table_5,table_6,table_7
    
    try:
        table_1=get_data_TimeSeries_KeyIndicators(file)
    except:
        print("Key Indicators Failed")
        pass
    
    try:
        table_2=get_data_TimeSeries_Action_Screen(file)
    except:
        print("Action Screen Failed")
        pass
    
    try:
        table_3=get_data_TimeSeries_Action_Call(file)
    except:
        print("Action Call Failed")
        pass
    
    try:
        table_4=get_data_TimeSeries_COVID_News(file)
    except:
        print("COVID News Failed")
        pass
    
    try:
        table_5=get_data_TimeSeries_NIH_Response(file)
    except:
        print("NIH Response Failed")
        pass

    try:
        table_6=get_data_TimeSeries_NIH_Risk(file)
    except:
        print("NIH Risk Failed")
        pass
    
    try:
        table_7=get_data_TimeSeries_Province_Response(file)
    except:
        print("Proving Response Failed")
        pass
    
    return table_1,table_2,table_3,table_4,table_5,table_6,table_7
    

In [82]:
start_date = date(2020, 3, 11)
end_date = date(2020, 3, 29)
Master_data_list=list()
for single_date in daterange(start_date, end_date):
    Date=single_date.strftime('%d-%b-%Y')
    print(Date)
    file = "C:/Users/Mehsam.Hemani/Downloads/COVID_NIH/COVID-19-Daily-Updated-SitRep-"+Date+".pdf"
    dataset=get_data(file)
#     print(dataset[0])
    Master_data_list.append(dataset)
#     .to_csv('COVID19_PAK_METRICS')

11-Mar-2020
12-Mar-2020
13-Mar-2020
14-Mar-2020
15-Mar-2020
16-Mar-2020
17-Mar-2020
18-Mar-2020
19-Mar-2020
20-Mar-2020
21-Mar-2020
22-Mar-2020
23-Mar-2020
24-Mar-2020
25-Mar-2020
26-Mar-2020
27-Mar-2020
28-Mar-2020


In [16]:
pd.concat([x[3] for x in Master_data_list]).groupby(["Date"])["News"].apply('\n '.join))

,News
Date,
11-Mar-2020,"Updates: Pakistan:\n Three new cases have been confirmed in the past 24 hours, bringing the tally to 19. •\n Number of calls to passengers returning from Iraq: 144 •\n Global:\n Total cases: 118,887, Total Deaths: 4,290 •\n Total affected countries: 109 including Turkey and Panama. •"
12-Mar-2020,"Updates: Pakistan:\n One case confirmed from GB, bringing the total to 20. •\n Number of calls to passengers returning from Italy: 71 •\n Global:\n Total cases: 125,860, Total Deaths: 4,619 •\n Total affected countries: 116 including Jordan and Congo •"
13-Mar-2020,"Updates: Pakistan:\n One case confirmed from Sindh, bringing the total to 21. •\n Number of calls to passengers returning from Italy: 40 •\n Global:\n Total cases: 133,138, Total Deaths: 4,983 •\n Total affected countries: 116 including Guyana •"
14-Mar-2020,"Updates: Pakistan:\n 7 new cases confirmed, bringing the tally to 28 •\n National Health Emergency declared •\n Global:\n Total cases: 144,633, Total Deaths: 5,438 •\n Total affected countries: 132 including Kenya and Sudan •"
15-Mar-2020,"Updates: Pakistan:\n 3 new cases confirmed, bringing the tally to 31. •\n All public gatherings, conferences and events banned till 5th April, 2020. •\n Global:\n Total cases:154,661, Total Deaths: 5,813 •\n Total affected countries: 137 including Curacao and Suriname •"
16-Mar-2020,"Updates: Pakistan:\n 22 new cases confirmed, bringing the tally to 53. •\n Global:\n Total cases:165,719, Total Deaths: 6,480 •\n Total affected countries: 142 including Uzbekistan and Seychelles •"
17-Mar-2020,"Updates: Pakistan:\n 134 new cases confirmed, bringing the tally to 187 •\n Calls to passengers from Italy: 207 •\n Global:\n Total cases:182,405, Total Deaths: 7,154 •\n Total affected countries: 150 including Greenland •"
18-Mar-2020,"Updates: Pakistan:\n 54 new cases confirmed, bringing the tally to 241 •\n Global:\n Total cases:198,004, Total Deaths: 7,948 •"
19-Mar-2020,"Updates: Pakistan:\n During last 24 hours, total 61 new cases confirmed by different laboratories, bringing the •\n national tally to 302 •\n Global:\n Total cases:218,631, Total Deaths: 8,809 •\n Total number of countries/regions affected: 157 •"


In [80]:
Date='24-Mar-2020'

file = "C:/Users/Mehsam.Hemani/Downloads/COVID_NIH/COVID-19-Daily-Updated-SitRep-"+Date+".pdf"

x=get_data_TimeSeries_NIH_Response(file)

# x['temp']=x.groupby(["Date","region"])["Provinces Preparedness and Response"].apply(' \n '.join)[0]
x

,Date,NIH Preparedness and Response,Agg NIH Preparedness and Response
0,24-Mar-2020,Contact tracing of confirmed cases being carried out by the RRTs. •,Contact tracing of confirmed cases being carried out by the RRTs. • \n Risk communication carried out among healthcare workers and affected communities during contact tracing. • \n NIH laboratory is providing support to provinces and regions for testing. •
1,24-Mar-2020,Risk communication carried out among healthcare workers and affected communities during contact tracing. •,Contact tracing of confirmed cases being carried out by the RRTs. • \n Risk communication carried out among healthcare workers and affected communities during contact tracing. • \n NIH laboratory is providing support to provinces and regions for testing. •
2,24-Mar-2020,NIH laboratory is providing support to provinces and regions for testing. •,Contact tracing of confirmed cases being carried out by the RRTs. • \n Risk communication carried out among healthcare workers and affected communities during contact tracing. • \n NIH laboratory is providing support to provinces and regions for testing. •


In [77]:
merge_temp=x.groupby(["Date","region"])["Provinces Preparedness and Response"].apply(' \n '.join)\
    .reset_index()\
    .rename(columns={'Provinces Preparedness and Response':'Agg Provinces Preparedness and Response'})
pd.merge(x,merge_temp,how='left', on=["Date","region"])

,region,Provinces Preparedness and Response,Date,Agg Provinces Preparedness and Response
0,Balochistan,Isolation and Quarantine facility in Dera Allahyar being updated and made functional. •,24-Mar-2020,Isolation and Quarantine facility in Dera Allahyar being updated and made functional. •
1,Sindh,Sindh Chief Minister meets with political leaders of different political parties. •,24-Mar-2020,Sindh Chief Minister meets with political leaders of different political parties. •
2,Punjab,Implementation on social distancing guidelines will start from 9am onwards in morning for next 14 days. •,24-Mar-2020,Implementation on social distancing guidelines will start from 9am onwards in morning for next 14 days. •
3,KP & TD,10 provincial officers from merged areas trained and deployed in the districts for monitoring COVID-19 response activities. •,24-Mar-2020,10 provincial officers from merged areas trained and deployed in the districts for monitoring COVID-19 response activities. •
4,AJK,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •",24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
5,GB,Fumigation of ambulances and isolation rooms are being carried out as per requirement. •,24-Mar-2020,Fumigation of ambulances and isolation rooms are being carried out as per requirement. •


In [53]:
x.groupby(["Date","region"])["Provinces Preparedness and Response"].apply(' \n '.join)

Date         region     
28-Mar-2020  AJK            A new field isolation hospital for COVID-19 with 50 bed capacity will be established in Muzaffarabad.  •  
             Balochistan    RRT teams are working  for contact tracing  •                                                             
             GB             All suspected Travellers coming from Taftan will be tested for COVID-19  •                                
             KP             Screeing teams are deputed at all district entry and exit points for screening of visitors of COVID -19  •
             Punjab         CM Punjab advice indoor stay and follow the guidelines issued by the Govt.   •                            
             Sindh          Distribution of N95 masks across the province was done   •                                                
Name: Provinces Preparedness and Response, dtype: object

In [15]:
pd.concat([x[0] for x in Master_data_list])

,Cumulative travelers Screened,Entry in last 24 hours,Suspected at Points of Entry,Travelers Screened at Points of,Travelers Screened at Points ofEntry in last 24 hours
11-Mar-2020,873288,NaN,26,NaN,21102.0
12-Mar-2020,894256,NaN,29,NaN,20968.0
13-Mar-2020,913542,NaN,33,NaN,19286.0
14-Mar-2020,933970,NaN,43,NaN,20428.0
15-Mar-2020,954646,NaN,59,NaN,20676.0
16-Mar-2020,975634,NaN,74,NaN,20988.0
17-Mar-2020,995821,NaN,95,NaN,20187.0
18-Mar-2020,1015909,NaN,115,NaN,20088.0
19-Mar-2020,1033884,NaN,151,NaN,17975.0
20-Mar-2020,1049282,NaN,170,NaN,15398.0


In [83]:
writer = pd.ExcelWriter('COVID_FINAL_DATA.xlsx', engine='xlsxwriter')

pd.concat([x[0] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_KeyIndicators')
pd.concat([x[1] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_Action_Screen')
pd.concat([x[2] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_Action_Call')
pd.concat([x[3] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_COVID_News')
pd.concat([x[4] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_NIH_Response')
pd.concat([x[5] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_NIH_Risk')
pd.concat([x[6] for x in Master_data_list]).to_excel(writer, sheet_name='TimeSeries_Province_Response')

writer.save()

,region,Provinces Preparedness and Response,Date,Agg Provinces Preparedness and Response
0,Balochistan,Isolation and Quarantine facility in Dera Allahyar being updated and made functional. •,24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
1,Sindh,Sindh Chief Minister meets with political leaders of different political parties. •,24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
2,Punjab,Implementation on social distancing guidelines will start from 9am onwards in morning for next 14 days. •,24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
3,KP & TD,10 provincial officers from merged areas trained and deployed in the districts for monitoring COVID-19 response activities. •,24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
4,AJK,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •",24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
6,GB,Fumigation of ambulances and isolation rooms are being carried out as per requirement. •,24-Mar-2020,"VTM kits made available at all designated hospitals to take samples of suspected cases being reported and sent to Virology Lab, NIH Islamabad. •"
